In [16]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from google.colab import drive
from tensorflow.data import AUTOTUNE

# Google Drive bağlantısı
drive.mount('/content/drive')

# CSV dosyasının yolu
csv_path = '/content/drive/MyDrive/hemorrhage_diagnosis.csv'
data = pd.read_csv(csv_path)

# Görüntü dosyalarının bulunduğu temel klasör
base_dir = '/content/drive/MyDrive/Patients_CT'

# Resim yollarını oluştur
def construct_image_path(row, base_dir):
    patient_folder = os.path.join(base_dir, f"{int(row['PatientNumber']):03}")
    image_path = os.path.join(patient_folder, 'brain', f"{int(row['SliceNumber'])}.jpg")
    return image_path

data['ImagePath'] = data.apply(lambda row: construct_image_path(row, base_dir), axis=1)
data = data[data['ImagePath'].apply(os.path.exists)]  # Yalnızca var olan dosyalar

# Etiketleri işleme
label_columns = ['Intraventricular', 'Intraparenchymal', 'Subarachnoid',
                 'Epidural', 'Subdural', 'No_Hemorrhage', 'Fracture_Yes_No']
multi_labels = data[label_columns].apply(lambda row: tuple(row.index[row == 1]), axis=1)
mlb = MultiLabelBinarizer()
one_hot_labels = mlb.fit_transform(multi_labels)

# Eğitim ve test setlerini bölme
train_data, test_data, train_labels, test_labels = train_test_split(
    data['ImagePath'], one_hot_labels, test_size=0.2, random_state=42
)

# Görüntü boyutu
IMG_SIZE = (128, 128)

# Veri ön işleme
def preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0  # Normalize
    return img

def preprocess_labels(labels):
    return tf.convert_to_tensor(labels, dtype=tf.float32)

# TensorFlow veri kümesini oluşturma
train_dataset = tf.data.Dataset.from_tensor_slices((list(train_data), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((list(test_data), test_labels))

def preprocess_labels(labels):
    return tf.cast(labels, dtype=tf.float32)

train_dataset = train_dataset.map(lambda x, y: (preprocess_image(x), preprocess_labels(y)), num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: (preprocess_image(x), preprocess_labels(y)), num_parallel_calls=AUTOTUNE)

# Batch ve cache işlemleri
train_dataset = train_dataset.shuffle(1000).batch(32).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)

# Model mimarisi
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_columns), activation='sigmoid')  # Çoklu etiketler için sigmoid aktivasyonu
])

# Modeli derleme
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Modelin özeti
model.summary()

# Modeli eğitme
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,
    verbose=1
)

# Modeli kaydetme
model.save('/content/drive/MyDrive/hemorrhage_model.h5')
print("Model başarıyla kaydedildi!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_39 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_40 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_41 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │      12,845,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,939,335 (49.36 MB)

 Trainable params: 12,939,335 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 332s 3s/step - accuracy: 0.7931 - loss: 0.2538 - val_accuracy: 0.8583 - val_loss: 0.1346
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.8375 - loss: 0.1444 - val_accuracy: 0.8822 - val_loss: 0.0965
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.8803 - loss: 0.0960 - val_accuracy: 0.8962 - val_loss: 0.0840
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8926 - loss: 0.0827 - val_accuracy: 0.8962 - val_loss: 0.0785
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.8958 - loss: 0.0616 - val_accuracy: 0.9022 - val_loss: 0.0813
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.9251 - loss: 0.0438 - val_accuracy: 0.9082 - val_loss: 0.0783
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9315 - loss: 0.0388 - val_accuracy: 0.9222 - val_loss: 0.0786
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.9424 - loss: 0.0362 - val_accuracy: 0.9142 - val_l

Model başarıyla kaydedildi!


In [19]:
# Modeli değerlendirme
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Test kaybı: {test_loss:.4f}")
print(f"Test doğruluğu: {test_accuracy:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 412ms/step - accuracy: 0.9394 - loss: 0.0907
Test kaybı: 0.1045
Test doğruluğu: 0.9242


In [20]:
# Modeli test etmek için tahminler
predictions = model.predict(test_dataset)

# İlk 5 tahminin üzerinden geçelim ve çıktı formatını istediğiniz gibi düzenleyelim
for i in range(20):
    # Gerçek etiketler
    true_label = test_labels[i]

    # Tahmin edilen etiket (en yüksek olasılığa sahip etiket)
    predicted_label = predictions[i]
    predicted_class = np.argmax(predicted_label)  # En yüksek olasılığı seç

    # Gerekli bilgileri alalım: PatientNumber ve SliceNumber
    patient_number = data.iloc[i]['PatientNumber']
    slice_number = data.iloc[i]['SliceNumber']

    # Gerçek etiket ve tahmin edilen etiketin düzenlenmesi
    true_label_single = [0] * len(label_columns)
    true_label_single[true_label.argmax()] = 1  # Gerçek etiketin doğru yerini 1 yap

    predicted_label_single = [0] * len(label_columns)
    predicted_label_single[predicted_class] = 1  # Tahmin edilen etiketin doğru yerini 1 yap

    # Sonuçları yazdırma
    print(f"Örnek {i+1}:")
    print(f"PatientNumber: {patient_number}, SliceNumber: {slice_number}")
    print(f"Gerçek etiket: {true_label_single}")
    print(f"Tahmin edilen etiket: {predicted_label_single}")
    print('-' * 40)


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 366ms/step
Örnek 1:
PatientNumber: 49, SliceNumber: 1
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 2:
PatientNumber: 49, SliceNumber: 2
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 3:
PatientNumber: 49, SliceNumber: 3
Gerçek etiket: [0, 1, 0, 0, 0, 0, 0]
Tahmin edilen etiket: [0, 1, 0, 0, 0, 0, 0]
----------------------------------------
Örnek 4:
PatientNumber: 49, SliceNumber: 4
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 5:
PatientNumber: 49, SliceNumber: 5
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 6:
PatientNumber: 49, SliceNumber: 6
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------

In [21]:
# Test veri seti üzerinde modelin tahminleri
predictions = model.predict(test_dataset)

# Test verisindeki her örneği değerlendirelim
for i in range(len(test_labels)):
    # Gerçek etiket
    true_label = test_labels[i]

    # Tahmin edilen etiket (en yüksek olasılığa sahip etiket)
    predicted_label = predictions[i]
    predicted_class = np.argmax(predicted_label)  # En yüksek olasılığı seç

    # Gerekli bilgileri alalım: PatientNumber ve SliceNumber
    patient_number = data.iloc[i]['PatientNumber']
    slice_number = data.iloc[i]['SliceNumber']

    # Gerçek etiket ve tahmin edilen etiketin düzenlenmesi
    true_label_single = [0] * len(label_columns)
    true_label_single[true_label.argmax()] = 1  # Gerçek etiketin doğru yerini 1 yap

    predicted_label_single = [0] * len(label_columns)
    predicted_label_single[predicted_class] = 1  # Tahmin edilen etiketin doğru yerini 1 yap

    # Sonuçları yazdırma
    print(f"Örnek {i+1}:")
    print(f"PatientNumber: {patient_number}, SliceNumber: {slice_number}")
    print(f"Gerçek etiket: {true_label_single}")
    print(f"Tahmin edilen etiket: {predicted_label_single}")
    print('-' * 40)


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 357ms/step
Örnek 1:
PatientNumber: 49, SliceNumber: 1
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 2:
PatientNumber: 49, SliceNumber: 2
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 3:
PatientNumber: 49, SliceNumber: 3
Gerçek etiket: [0, 1, 0, 0, 0, 0, 0]
Tahmin edilen etiket: [0, 1, 0, 0, 0, 0, 0]
----------------------------------------
Örnek 4:
PatientNumber: 49, SliceNumber: 4
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 5:
PatientNumber: 49, SliceNumber: 5
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------------------------------------
Örnek 6:
PatientNumber: 49, SliceNumber: 6
Gerçek etiket: [0, 0, 0, 0, 1, 0, 0]
Tahmin edilen etiket: [0, 0, 0, 0, 1, 0, 0]
----------

In [27]:
def find_wrong_predictions(model, test_dataset, test_labels, data):
    wrong_predictions = []
    predictions = model.predict(test_dataset)

    for i in range(len(test_labels)):
        true_label = test_labels[i]
        predicted_label = predictions[i]
        predicted_class = np.argmax(predicted_label)

        # Gerçek etiketin ve tahmin edilen etiketin karşılaştırılması
        true_label_single = [0] * len(true_label)
        true_label_single[true_label.argmax()] = 1

        predicted_label_single = [0] * len(predicted_label)
        predicted_label_single[predicted_class] = 1

        if not np.array_equal(true_label_single, predicted_label_single):
            wrong_predictions.append(i + 1)  # İndeks 1'den başlamalı

    return wrong_predictions

# Yanlış tahminlerin listesini almak için fonksiyonu çağırın
wrong_predictions = find_wrong_predictions(model, test_dataset, test_labels, data)
print("Yanlış tahmin yapılan örnekler:", wrong_predictions)


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 359ms/step
Yanlış tahmin yapılan örnekler: [10, 30, 48, 79, 97, 105, 133, 151, 173, 180, 194, 196, 197, 256, 271, 277, 287, 292, 294, 303, 307, 311, 351, 357, 385, 390, 392, 396, 404, 425, 445, 451, 470, 471, 482, 486, 489, 499]
